# Introduction to the Planner

The Planner is one of the fundamental concepts of the Semantic Kernel.

It makes use of the collection of native and semantic functions that have been registered to the kernel and using AI, will formulate a plan to execute the given ask.

From our own testing, planner works best with more powerful models like `gpt4` but sometimes you might get working plans with cheaper models like `gpt-35-turbo`. We encourage you to implement your own versions of the planner and use different models that fit your user needs.  

Read more about planner [here](https://aka.ms/sk/concepts/planner)

In [1]:
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import (
    AzureChatCompletion,
)
import os
from dotenv import load_dotenv

load_dotenv()
kernel = sk.Kernel()

# Configure AI service used by the kernel
kernel.add_chat_service(  # We are adding a text service
    "AzureOpenAI",  # The alias we can use in prompt templates' config.json
    AzureChatCompletion(
        deployment_name=os.getenv("CHAT_COMPLETION_NAME"),  # Azure OpenAI *Deployment name*
        endpoint=os.getenv("OPENAI_API_BASE"),  # Azure OpenAI *Endpoint*
        api_key=os.getenv("OPENAI_API_KEY"),  # Azure OpenAI *Key*
    ),
)


## It all begins with an ask

In [2]:
ask = """
Tomorrow is Valentine's day. I need to come up with a few date ideas. She speaks French so write it in French.
Convert the text to uppercase"""

### Providing skills to the planner
The planner needs to know what skills are available to it. Here we'll give it access to the `SummarizeSkill` and `WriterSkill` we have defined on disk. This will include many semantic functions, of which the planner will intelligently choose a subset. 

You can also include native functions as well. Here we'll add the TextSkill.

In [3]:
from semantic_kernel.core_skills.text_skill import TextSkill

skills_directory = "./skills/"
summarize_skill = kernel.import_semantic_skill_from_directory(skills_directory, "SummarizeSkill")
writer_skill = kernel.import_semantic_skill_from_directory(skills_directory, "WriterSkill")
text_skill = kernel.import_skill(TextSkill(), "TextSkill")

Define your ASK. What do you want the Kernel to do?

# Basic Planner

 Let's start by taking a look at a basic planner. The `BasicPlanner` produces a JSON-based plan that aims to solve the provided ask sequentially and evaluated in order.

In [4]:
from semantic_kernel.planning.basic_planner import BasicPlanner
planner = BasicPlanner()

In [5]:
basic_plan = await planner.create_plan_async(ask, kernel)

In [6]:
print(basic_plan.generated_plan)

{
  "input": "Valentine's Day Date Ideas",
  "subtasks": [
    {"function": "WriterSkill.Brainstorm"},
    {"function": "WriterSkill.Translate", "args": {"language": "French"}},
    {"function": "TextSkill.uppercase"}
  ]
}


You can see that the Planner took my ask and converted it into an JSON-based plan detailing how the AI would go about solving this task, making use of the skills that the Kernel has available to it.

As you can see in the above plan, the AI has determined which functions to call in order to fulfill the user ask. The output of each step of the plan becomes the input to the next function.

Let's also define an inline skill and have it be available to the Planner. Be sure to give it a function name and skill name.

In [7]:
sk_prompt = """
{{$input}}

Rewrite the above in the style of Shakespeare.
"""
shakespeareFunction = kernel.create_semantic_function(sk_prompt, "shakespeare", "ShakespeareSkill",
                                                      max_tokens=2000, temperature=0.8)

Let's update our ask using this new skill

In [8]:
ask = """
Tomorrow is Valentine's day. I need to come up with a few date ideas.
She likes Shakespeare so write using his style. She speaks French so write it in French.
Convert the text to uppercase."""

new_plan = await planner.create_plan_async(ask, kernel)

In [9]:
print(new_plan.generated_plan)

{
    "input": "Valentine's Day Date Ideas",
    "subtasks": [
        {"function": "WriterSkill.Brainstorm"},
        {"function": "ShakespeareSkill.shakespeare"},
        {"function": "WriterSkill.Translate", "args": {"language": "French"}},
        {"function": "TextSkill.uppercase"}
    ]
}


### Executing the plan

Now that we have a plan, let's try to execute it! The Planner has a function called `execute_plan`.

In [10]:
results = await planner.execute_plan_async(new_plan, kernel)

In [11]:
print(results)

1. UN PIQUE-NIQUE JUSTE AU MILIEU DU PARC VERDOYANT
2. ENSEMBLE, NOUS CUISINERONS UN FESTIN ROMANTIQUE
3. LAISSEZ-NOUS ERRER À TRAVERS LES SENTIERS PITTORESQUES
4. AVEC DU VIN À LA MAIN, NOUS PEIGNONS AVEC L'ÉTREINTE DE L'AMOUR
5. UN ROMANCE CLASSIQUE, VUE DANS NOTRE MAISON
6. À LA GALERIE D'ART OU AU MUSÉE, NOUS IRONS
7. UN COURS DE DANSE, OÙ NOUS BOUGERONS COMME UN SEUL
8. SUR UN BATEAU, NOUS NAVIGUERONS ALORS QUE LE JOUR DÉCLINE
9. UN SPECTACLE DE MUSIQUE OU DE COMÉDIE EN DIRECT
10. UNE JOURNÉE AU SPA, POUR QUE LES AMOUREUX EN PROFITENT TOUS LES DEUX


# The Plan Object Model

To build more advanced planners, we need to introduce a proper Plan object that can contain all the necessary state and information needed for high quality plans.

To see what that object model is, look at (https://github.com/microsoft/semantic-kernel/blob/main/python/semantic_kernel/planning/plan.py)

# Sequential Planner

The sequential planner is an XML-based step-by-step planner. You can see the prompt used for it here (https://github.com/microsoft/semantic-kernel/blob/main/python/semantic_kernel/planning/sequential_planner/Skills/SequentialPlanning/skprompt.txt)

In [12]:
from semantic_kernel.planning import SequentialPlanner
planner = SequentialPlanner(kernel)

In [13]:
sequential_plan = await planner.create_plan_async(goal=ask)

To see the steps that the Sequential Planner will take, we can iterate over them and print their descriptions

In [14]:
for step in sequential_plan._steps:
    print(step.description, ":", step._state.__dict__)

Given a goal or topic description generate a list of ideas : {'variables': {'input': ''}}
Turn a scenario into a short and entertaining poem. : {'variables': {'input': ''}}
Translate the input into a language of your choice : {'variables': {'input': ''}}
Convert a string to uppercase. : {'variables': {'input': ''}}


Let's ask the sequential planner to execute the plan.

In [15]:
result = await sequential_plan.invoke_async()

In [16]:
print(result)

LES ROSES SONT ROUGES, LES VIOLETTES SONT BLEUES
J'AI ÉCRIT CE POÈME JUSTE POUR TOI
MAIS SOYONS RÉALISTES, JE NE SUIS PAS SI RUSÉ
ALORS VOICI DU CHOCOLAT, ESSAYONS!


# Action Planner

The action planner takes in a list of functions and the goal, and outputs a **single** function to use that is appropriate to meet that goal.

In [17]:
from semantic_kernel.planning import ActionPlanner
planner = ActionPlanner(kernel)

Let's add more skills to the kernel

In [18]:
from semantic_kernel.core_skills import FileIOSkill, MathSkill, TextSkill, TimeSkill
kernel.import_skill(MathSkill(), "math")
kernel.import_skill(FileIOSkill(), "fileIO")
kernel.import_skill(TimeSkill(), "time")
kernel.import_skill(TextSkill(), "text")

{'lowercase': <semantic_kernel.orchestration.sk_function.SKFunction at 0x1ac997f70d0>,
 'trim': <semantic_kernel.orchestration.sk_function.SKFunction at 0x1ac9983a710>,
 'trim_end': <semantic_kernel.orchestration.sk_function.SKFunction at 0x1ac998392a0>,
 'trim_start': <semantic_kernel.orchestration.sk_function.SKFunction at 0x1ac99839c00>,
 'uppercase': <semantic_kernel.orchestration.sk_function.SKFunction at 0x1ac99839540>}

In [19]:
ask = "What is the sum of 110 and 990?"

In [20]:
plan = await planner.create_plan_async(goal=ask)

In [21]:
result = await plan.invoke_async()

In [22]:
print(result)

1100


# Stepwise Planner

Stepwise Planner is based off the paper from MRKL (Modular Reasoning, Knowledge and Language) and is similar to other papers like ReACT (Reasoning and Acting in Language Models). At the core, the stepwise planner allows for the AI to form "thoughts" and "observations" and execute actions based off those to achieve a user's goal. This continues until all required functions are complete and a final output is generated.

See a video walkthrough of Stepwise Planner [here.](https://youtu.be/DG_Ge1v0c4Q?si=T1CHaAm1vV0mWRHu)

In [23]:
from semantic_kernel.planning import StepwisePlanner
from semantic_kernel.planning.stepwise_planner.stepwise_planner_config import (
    StepwisePlannerConfig,
)

Let's create a Bing Search native skill that we can pass in to the Kernel.

Make sure you have a Bing Search API key in your `.env` file

(https://www.microsoft.com/en-us/bing/apis/bing-web-search-api)

In [33]:
class WebSearchEngineSkill:
    """
    A search engine skill.
    """
    from semantic_kernel.orchestration.sk_context import SKContext
    from semantic_kernel.skill_definition import sk_function, sk_function_context_parameter

    def __init__(self, connector) -> None:
        self._connector = connector

    @sk_function(
        description="Performs a web search for a given query", name="searchAsync"
    )
    @sk_function_context_parameter(
        name="query",
        description="The search query",
    )
    async def search_async(self, query: str, context: SKContext) -> str:
        query = query or context.variables.get("query")
        result = await self._connector.search_async(query, num_results=5, offset=0)
        return str(result)

In [34]:
from semantic_kernel.connectors.search_engine import BingConnector

BING_API_KEY = os.getenv("BING_SUBSCRIPTION_KEY")
connector = BingConnector(BING_API_KEY)
kernel.import_skill(WebSearchEngineSkill(connector), skill_name="WebSearch")

{'searchAsync': <semantic_kernel.orchestration.sk_function.SKFunction at 0x1ac99894280>}

Let's also add a couple more skills

In [35]:
from semantic_kernel.core_skills.math_skill import MathSkill
from semantic_kernel.core_skills.time_skill import TimeSkill

kernel.import_skill(TimeSkill(), "time")
kernel.import_skill(MathSkill(), "math")

{'Add': <semantic_kernel.orchestration.sk_function.SKFunction at 0x1ac998b5cc0>,
 'Subtract': <semantic_kernel.orchestration.sk_function.SKFunction at 0x1ac998b6170>}

In [36]:
planner = StepwisePlanner(
    kernel, StepwisePlannerConfig(max_iterations=10, min_iteration_time_ms=1000)
)

Now let's do a more complicated ask that will require planner to make a call to Bing to get the latest information.

In [37]:
ask = """How many total championships combined do the top 5 teams in the NBA have?"""

plan = planner.create_plan(goal=ask)

In [38]:
result = await plan.invoke_async()

In [39]:
print(result)

The top 5 teams in the NBA have a combined total of 34 championships.


Let's see the steps that the AI took to get to the answer.

In [40]:
for index, step in enumerate(plan._steps):
    print("Step:", index)
    print("Description:",step.description)
    print("Function:", step.skill_name + "." + step._function.name)
    if len(step._outputs) > 0:
        print( "  Output:\n", str.replace(result[step._outputs[0]],"\n", "\n  "))

Step: 0
Description: Execute a plan
Function: StepwisePlanner.ExecutePlan
  Output:
 This was my previous work (but they haven't seen any of it! They only see what I return as final answer):
  [THOUGHT]
  To answer this question, I need to find the total number of championships won by the top 5 teams in the NBA. I can do this by searching for the championship count of each team and then adding them together.
  [ACTION]
  {"action": "WebSearch.searchAsync", "action_variables": {"query": "NBA top 5 teams total championships count"}}
  [OBSERVATION]
  ['Having defeated Miami Heat 4-2 in the 2020 NBA Finals, the Los Angeles Lakers clinched their 17th NBA championship, drawing level with the Boston Celtics for the most titles of all time. The...', "The Celtics and Lakers are tied for the most championships in NBA history with 17 each. Last season, the Warriors won the franchise's seventh title, defeating the Celtics in the 2022 NBA...", 'Celtics and Lakers are the teams with most championsh